In [2]:
import tensorflow as tf               # TesorFlow
import numpy as np                    # NumPy
import matplotlib.pyplot as plt       # Графики-то строить будем
import pandas as pd                   # Для работы с таблицами и CSV-файлами
import sklearn.metrics                # Для построения confusion matrix
import seaborn as sns                 # Для отображения confusion matrix
import os                             # Для возможности выбора GPU

# Библиотека для упрощения работы с SVD
# В колабе ее можно поставить командой !pip install git+https://github.com/mayukh18/reco.git
import reco.recommender

In [3]:
# Используем GPU 0
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Отключаем отладочные сообщения в TensorFlow
tf.get_logger().setLevel('ERROR')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "2"

In [5]:
df = pd.read_csv('train.csv')
df

,user_id,course_id
0,39972,34
1,56815,51
2,63734,20
3,17896,81
4,36961,64
...,...,...
10267,10520,10
10268,62231,40
10269,150361,148
10270,4182,7


In [10]:
df['num'] = 1
df

,user_id,course_id,num
0,39972,34,1
1,56815,51,1
2,63734,20,1
3,17896,81,1
4,36961,64,1
...,...,...,...
10267,10520,10,1
10268,62231,40,1
10269,150361,148,1
10270,4182,7,1


In [11]:
utility_matrix = df.pivot(index="user_id", columns="course_id", values='num') #.fillna(0)
utility_matrix

course_id,1,3,5,6,7,8,9,10,12,13,...,225,227,228,229,234,235,237,238,246,252
user_id,,,,,,,,,,,,,,,,,,,,,
4,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
186668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
186835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [16]:
svd = reco.recommender.SVDRecommender(no_of_features=8)

In [17]:
svd.fit(utility_matrix.to_numpy(), utility_matrix.index.values.tolist(), list(utility_matrix.columns))

In [26]:
df2 = pd.read_csv('sample_submission.csv')
df2

,user_id,course_id_1,course_id_2,course_id_3
0,78,0,0,0
1,81,0,0,0
2,120,0,0,0
3,123,0,0,0
4,150,0,0,0
...,...,...,...,...
2895,185864,0,0,0
2896,186262,0,0,0
2897,186691,0,0,0
2898,186887,0,0,0


In [42]:
most_popular = [1, 2, 3]             # ПОТОМ ИСПРАВИТЬ!!!!!

In [65]:
user_list = list(df2['user_id'])
existed_users = list(df['user_id'])
a = []
b = []
c = []
for u_id in user_list:
    #print(u_id)
    if u_id in existed_users:
        u_id_recom = svd.recommend([u_id], N=3)[0]
    else:
        u_id_recom = most_popular
    #print(u_id_recom)
    course_id_1_recom = (u_id_recom)[0]
    a.append(course_id_1_recom)
    course_id_2_recom = (u_id_recom)[1]
    b.append(course_id_2_recom)
    course_id_3_recom = (u_id_recom)[2]
    c.append(course_id_3_recom)
df2['course_id_1'] = a
df2['course_id_2'] = b
df2['course_id_3'] = c
df2

#svd.recommend([user_id], N=3)
#df.insert(4, 'recomend', df['user_id'].apply())
#df

,user_id,course_id_1,course_id_2,course_id_3
0,78,252,86,97
1,81,252,87,99
2,120,252,86,97
3,123,1,2,3
4,150,252,86,97
...,...,...,...,...
2895,185864,1,2,3
2896,186262,1,2,3
2897,186691,1,2,3
2898,186887,1,2,3


In [73]:
df2.to_csv('submission-matvey.csv', sep=',', index=False, columns=['user_id', 'course_id_1', 'course_id_2', 'course_id_3'])